# How to use this notebook
This notebook is meant to accompany an article published in "Linux Magazin" XXXXX (german). The code snippets directly refer to the article are probably not self-explanatory.

### _Vorbereitungen_
Imports der benötigten Bibliotheken, API einrichten

In [1]:
from qiskit import QuantumCircuit, execute,  Aer, QuantumRegister, IBMQ
from qiskit.visualization import *
from qiskit.circuit.library import QFT
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
from qiskit_aqt_provider import AQTProvider
from qiskit.visualization import plot_histogram
from qiskit.compiler import transpile, assemble

In [2]:
from qiskit.providers.ibmq import least_busy

IBMQ_API_TOKEN = "PUT_YOUR_IBMQ_TOKEN_HERE"
AQT_TOKEN = "Put AQT Token here"
IBMQ.enable_account(IBMQ_API_TOKEN)
provider = IBMQ.get_provider()
device = least_busy(provider.backends(simulator=True))

RequestsApiError: '401 Client Error: Unauthorized for url: https://auth.quantum-computing.ibm.com/api/users/loginWithToken. Login failed., Error code: 3446.'

### 4. Erste Schritte / Praxisbeispiele

In [ ]:
simulator = Aer.get_backend('qasm_simulator')

qc1 = QuantumCircuit(3)
qc1.h(0)
qc1.cx(0, 1)
qc1.cx(0, 2)
qc1.measure_all()

# Schaltkreis mit Aer simulieren
job = execute(qc1, simulator, shots=4000)
result = job.result()
counts=result.get_counts()
print(counts)

In [ ]:
# Schaltkreis auf Quantencomputer laufen lassen

job = execute(qc1, backend = device, shots=4000)
result = job.result()
counts=result.get_counts()
print(counts)

#### Lokales Beispiel mit Qiskit Runtime

In [ ]:
N = 5
qc2 = QuantumCircuit(N)
qc2.h(0)
for x in range(1, N):
  qc2.cx(0,x)
#qc2 += QFT(num_qubits=N)
qc2.compose(QFT(num_qubits=N), inplace=True)
qc2.measure_all()
program_inputs = {
    "circuits": qc2,
    "circuit_indices": [0],
}
service = QiskitRuntimeService(channel="ibm_quantum", token=IBMQ_API_TOKEN)
options = {'backend_name': "ibmq_qasm_simulator"}
job = service.run(
    program_id="sampler",
    options=options,
    inputs=program_inputs,
)

#Dieser Teil ist blockierend. Man wartet auf das Ergebnis.
print(job.result())

#### AQT

In [ ]:
aqt = AQTProvider(AQT_TOKEN)
print(aqt.backends())
sim_backend = aqt.backends.aqt_qasm_simulator

### Error Mitigation

In [ ]:
# Schaltkreis mit Aer simulieren

counts = execute(qc1, simulator, shots=10000).result().get_counts()
plot_histogram(counts)

In [ ]:
# Schaltkreis auf Quantencomputer laufen lassen

device = provider.get_backend('ibmq_belem')
results = execute(qc1, device, shots=10000).result()
noisy_counts = results.get_counts()
print(noisy_counts)

# Beispielhaftes Ergebnis: {'000': 4973, '001': 70, '010': 145, '011': 334, '100': 55, '101': 254, '110': 173, '111': 3996}

plot_histogram(noisy_counts)

In [ ]:
# Deprecated Library was updated for this notebook to work
#from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
from qiskit.utils.mitigation import complete_meas_cal,CompleteMeasFitter

In [ ]:
# Durchlaufe alle acht Basiszustände

qr = QuantumRegister(3)
meas_calibs, state_labels = complete_meas_cal(qr=qr, circlabel='mcal')

for circuit in meas_calibs:
    print('Circuit',circuit.name)
    print(circuit)
    print()

In [ ]:
# Berechne Korrekturmatrix

t_qc = transpile(meas_calibs, device)
cal_results = device.run(t_qc, shots=10000).result()

meas_fitter = CompleteMeasFitter(cal_results, state_labels, circlabel='mcal')
array_to_latex(meas_fitter.cal_matrix)

In [ ]:
# Filter Objekt erstellen
meas_filter = meas_fitter.filter

# Neue Ergebnisse nach Mitigation
mitigated_results = meas_filter.apply(results)
mitigated_counts = mitigated_results.get_counts()

noisy_counts = results.get_counts()
plot_histogram([noisy_counts, mitigated_counts], legend=['noisy', 'mitigated'])

In [ ]:
# Berechnung der Quasi-Probabilities mit IBMQ-QASM-Simulator

service = QiskitRuntimeService()

with Sampler(circuits=[qc1], service=service, options={ "backend": "ibmq_qasm_simulator" }) as sampler:
    result = sampler(circuits=[0], shots=10000)
    print(result)

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table